---
title: "Téléchargement et traitement des données de rivières"
---

# Imports

In [ ]:
import requests
from zipfile import ZipFile
import os
from tqdm import tqdm
import geopandas as gpd

# Téléchargement, dézippage et geoparquet

In [22]:
def download_file(url: str, filename: str) -> None:
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get("content-length", 0))
    block_size = 1024
    progress_bar = tqdm(total=total_size, unit="B", unit_scale=True)

    with open(filename, "wb") as f:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            f.write(data)

    progress_bar.close()
    print(f"📥 Téléchargement terminé : {filename}")


def decompress_zip(input_file: str, output_file: str) -> None:
    with ZipFile(input_file, "r") as zip_ref:
        zip_ref.extractall(output_file)

    os.remove(input_file)
    print(f"📂 Décompression terminée : {output_file}")

In [7]:
url = "https://services.sandre.eaufrance.fr/telechargement/geo/ETH/BDTopage/2024/CoursEau/CoursEau_FXX-geojson.zip"

In [ ]:
download_file(url, filename="CoursEau_geojson.zip")

100%|██████████| 399M/399M [01:16<00:00, 5.21MB/s] 

📥 Téléchargement terminé : CoursEau_geojson.zip


In [ ]:
decompress_zip(input_file="CoursEau_geojson.zip", output_file="CoursEau")

📂 Décompression terminée : CoursEau.geojson


On lit les données, on s'assure qu'elles ont le bon CRS et on écrit ça en Geoparquet pour optimiser les opérations de lecture et jointure qui vont suivre.


In [ ]:
data = gpd.read_file("CoursEau/CoursEau_FXX.geojson")

In [44]:
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
(
    gpd.read_file("CoursEau/CoursEau_FXX.geojson")
    .set_geometry("geometry")
    .to_crs("epsg:4326")
    .to_parquet("CoursEau.geoparquet")
)

# Lecture du Geoparquet

In [46]:
data = gpd.read_parquet("CoursEau.geoparquet")

Ici seulement deux variables nous intéressent: le Nom de la rivière (pas mal de données manquantes) et sa géométrie.

In [ ]:
data = data[["TopoOH", "geometry"]]

TopoOH  \
0                           None   
1       Ruisseau de Saint-Blaise   
2                           None   
3          Ruisseau de Chemisard   
4                           None   
...                          ...   
142398                      None   
142399                      None   
142400                      None   
142401        Ruisseau des Morts   
142402                      None   

                                                 geometry  
0       MULTILINESTRING ((2.0475 45.05378, 2.04739 45....  
1       MULTILINESTRING ((2.18329 43.14797, 2.1832 43....  
2       MULTILINESTRING ((2.05402 45.0513, 2.05402 45....  
3       MULTILINESTRING ((5.65572 44.80644, 5.65577 44...  
4       MULTILINESTRING ((2.04054 45.11237, 2.04061 45...  
...                                                   ...  
142398  MULTILINESTRING ((2.03156 45.10487, 2.0314 45....  
142399  MULTILINESTRING ((2.05311 45.07831, 2.05316 45...  
142400  MULTILINESTRING ((2.04118 45.17135, 2.0408 45....  
142401  MULTILINESTRING ((5.70476 44.82948, 5.70484 44...  
142402  MULTILINESTRING ((2.02854 45.1528, 2.02842 45....  

[142403 rows x 2 columns]